In [2]:
import numpy as np
import json

In [5]:
gpu_lists = ['1080', '3090']
batch_sizes = ['16', '32']
models = ['alexnet', 'resnet34']

trace_files = []
for gpu in gpu_lists:
    for batch in batch_sizes:
        for model in models:
            trace_files.append(f'{gpu}-batch-{batch}-{model}-trace.json')

for trace in trace_files:
    print(trace)

1080-batch-16-alexnet-trace.json
1080-batch-16-resnet34-trace.json
1080-batch-32-alexnet-trace.json
1080-batch-32-resnet34-trace.json
3090-batch-16-alexnet-trace.json
3090-batch-16-resnet34-trace.json
3090-batch-32-alexnet-trace.json
3090-batch-32-resnet34-trace.json


In [20]:
with open('./1080-batch-32-alexnet-trace.json') as json_file:
    json_data = json.load(json_file)

    for key in json_data.keys():
        print(key)

    events = json_data['traceEvents']

    cuda_events = []
    for event in events:
        if 'stream' in str(event['tid']) and 'name' in event.keys() and 'dur' in event.keys():
            cuda_events.append(event)

    kernel_names = []
    for event in cuda_events:
        kernel_names.append(event['name'])

    kernel_names = np.unique(kernel_names)
    for name in kernel_names:
        print(name)

    

schemaVersion
deviceProperties
traceEvents
Memcpy HtoD (Pageable -> Device)
Memset (Device)
cask_cudnn::computeOffsetsKernel(cask_cudnn::ComputeOffsetsParams)
maxwell_scudnn_128x32_relu_large_nn_v1
maxwell_scudnn_winograd_128x128_ldg1_ldg4_relu_tile418n_nt_v1
maxwell_sgemm_128x64_nn
sgemm_32x32x32_NT_vec
void at::native::(anonymous namespace)::adaptive_average_pool<float>(float*, float*, int, int, int, int, long, long, long)
void at::native::(anonymous namespace)::max_pool_forward_nchw<float, float>(int, float const*, int, int, int, int, int, int, int, int, int, int, int, int, int, int, float*, long*)
void at::native::unrolled_elementwise_kernel<at::native::AddFunctor<float>, at::detail::Array<char*, 3>, OffsetCalculator<2, unsigned int>, OffsetCalculator<1, unsigned int>, at::native::memory::LoadWithoutCast, at::native::memory::StoreWithoutCast>(int, at::native::AddFunctor<float>, at::detail::Array<char*, 3>, OffsetCalculator<2, unsigned int>, OffsetCalculator<1, unsigned int>, at::na